In [12]:
!pip install streamlit pyngrok --quiet

In [21]:
!pkill -f ngrok

In [22]:
!ngrok authtoken 2yD7JZWRLeAgDLmOWlY3Vi9b0zv_6NFgPaLQxKbNsCcuJTTmV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [26]:
%%writefile "Smart_app.py"
import streamlit as st
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
    model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
    instruct_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return instruct_pipeline

model = load_model()

st.title("SmartSDLC - AI-enhanced Software Development Life Cycle")
st.write("✅ This app is running from Google Colab using Streamlit + ngrok!")

menu = ["Requirement Analysis", "Code Generation", "Code Review", "Test Case Generation"]
choice = st.sidebar.selectbox("Select Stage", menu)

def generate_response(prompt, max_tokens=200):
    output = model(prompt, max_new_tokens=max_tokens, do_sample=False)[0]['generated_text']
    return output.replace(prompt, "").strip()

if choice == "Requirement Analysis":
    st.header("Requirement Analysis & Summarization")
    req_text = st.text_area("Paste your software requirements here:")
    if st.button("Summarize Requirements"):
        if req_text.strip():
            prompt = f"Summarize the following software requirement:\n\n{req_text}\n\nSummary:"
            summary = generate_response(prompt, max_tokens=100)
            st.success("Summary:")
            st.write(summary)
        else:
            st.warning("Please input requirements text.")

elif choice == "Code Generation":
    st.header("Generate Code from Requirements")
    req_text = st.text_area("Describe the functionality you want to implement:")
    if st.button("Generate Code"):
        if req_text.strip():
            prompt = f"Generate Python code for the following functionality:\n\n{req_text}\n\nPython code:"
            code = generate_response(prompt, max_tokens=150)
            st.code(code, language="python")
        else:
            st.warning("Please input a description.")

elif choice == "Code Review":
   st.header("Automated Code Review")
   code = st.text_area("Paste your code here for review:")
   if st.button("Review Code"):
        if code.strip():
            prompt = f"Review the following Python code and list any issues or improvements:\n\n{code}\n\nReview:"
            review = generate_response(prompt, max_tokens=150)
            st.warning("Review Comments:")
            st.write(review)
        else:
            st.warning("Please paste code to review.")

elif choice == "Test Case Generation":
    st.header("Generate Test Cases from Requirements")
    req_text = st.text_area("Paste the functionality or requirements:")
    if st.button("Generate Test Cases"):
        if req_text.strip():
            prompt = f"Based on the following requirements, generate a list of software test cases:\n\n{req_text}\n\nTest Cases:"
            cases = generate_response(prompt, max_tokens=150)
            st.write("Suggested Test Cases:")
            st.write(cases)
        else:
            st.warning("Please input requirements.")


Writing Smart_app.py


In [27]:
from pyngrok import ngrok
import os

# Run Streamlit app in background
os.system("streamlit run Smart_app.py --server.port 8501 &")

# Wait a bit for the app to start
import time
time.sleep(5)

# Open ngrok tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print("🚀 Your SmartSDLC app is live at:", public_url)


🚀 Your SmartSDLC app is live at: NgrokTunnel: "https://dd40-104-196-97-191.ngrok-free.app" -> "http://localhost:8501"
